In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode('Its really bad', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[ 1.0002,  1.5748,  1.2460, -0.8462, -2.3700]],
       grad_fn=<AddmmBackward>)

In [ ]:
int(torch.argmax(result.logits))+1

1

In [ ]:
r = requests.get('https://www.yelp.com/biz/jamies-place-san-francisco')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [ ]:
reviews

["This place is legit the best breakfast place in San Francisco. I was blown away this afternoon with my Pork Benedict. All the flavors worked together so well. I was basically licking my plate, that's how great it was. My friend got the Denver Omelette and the eggs were extremely fluffy and perfect. Not to mention, the service here is phenomenal! Seriously, don't pass this place up.",
 "Came on a Saturday morning at 10am and was seated right away. People started to trickle in after we have arrived and the place quickly filled up at around 10:45am. Despite this cute quaint breakfast shop, the food did not live up to expectations. The presentation of the dishes were very nice, but the taste was disappointing. I ordered the California omelette and the egg was bland and spongey. It came with chicken apple sausages, green onions, garlic, tomatoes, avocado, and cheddar cheese. I asked for no tomatoes, but when the dish came...it had so many green onions I couldn't taste anything else. The d

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [ ]:
df.tail()

,review
5,November 6th 2021 10:30am I came here on a Sat...
6,My partner and I landed in San Francisco late ...
7,My second time coming here and this is a great...
8,I am obsessed with this mom and pop style rest...
9,we came in here on a whim and had a great expe...


In [ ]:
df['review'].iloc[2]


"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"

In [ ]:
def get_feel(value):
  return {
        1: "really bad",
        2: "not good",
        3: "ok",
        4: "good",
        5: "great",
    }[value]
    

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[2])


5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: get_feel(sentiment_score(x[:512])))


In [ ]:
df

,review,sentiment
0,This place is legit the best breakfast place i...,great
1,Came on a Saturday morning at 10am and was sea...,not good
2,First time here and it was pretty good. I had ...,good
3,We had lunch here and loved everything about t...,great
4,Wondered on Yelp to check others reviews for a...,not good
5,November 6th 2021 10:30am I came here on a Sat...,ok
6,My partner and I landed in San Francisco late ...,great
7,My second time coming here and this is a great...,great
8,I am obsessed with this mom and pop style rest...,good
9,we came in here on a whim and had a great expe...,great
